In [3]:
import time
import requests
import pandas as pd
import plotly.express as px

from IPython.display import clear_output

ISS_URL = 'https://api.wheretheiss.at/v1/satellites/25544'
ORBIT_TIME_SECS = 5_520  # Time required for ~1 complete orbit of ISS.


def get_iss_telemetry(url):
    """Return DataFrame of current ISS telemetry from URL."""
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch ISS position from {url}. \
        Status code: {response.status_code}")
    data = response.json()
    telemetry = pd.DataFrame(data, index=[0])
    telemetry = telemetry.drop(['id', 'footprint', 'daynum', 
                              'solar_lat', 'solar_lon'], axis=1)
    return telemetry


def track_iss(url, num_orbits=2, interval=3):
    """
    Plot current ISS location from URL and record and plot its track.
    
    Arguments:
       url = ISS telemetry URL -> string
       num_orbits = Number of ISS orbits to plot -> integer
       interval = Wait period (seconds) between calls to URL -> integer
   """
    
    num_pulls = int(ORBIT_TIME_SECS * num_orbits / interval)
    latitudes = []
    longitudes = []
    
    for _ in range(num_pulls):
        df = get_iss_telemetry(url).round(2)
        latitudes.append(df['latitude'].iloc[0])
        longitudes.append(df['longitude'].iloc[0])        
        clear_output(wait=True)
        
        fig = px.scatter_geo(df, 
                             lat='latitude', 
                             lon='longitude', 
                             color='visibility',
                             color_discrete_map = {'daylight': 'red',
                                                   'visible': 'orange',
                                                   'eclipsed': 'black'},
                             hover_data=['timestamp', 'altitude', 
                                         'velocity', 'units'])
        fig.add_trace(px.line_geo(lat=latitudes, lon=longitudes).data[0])
        fig.update_traces(marker=dict(symbol='x-open', size=10), 
                          line_color='blue')
        fig.update_layout(width=700, height=500, 
                          title='International Space Station Tracking')
        fig.show()
        
        time.sleep(interval)
        
        
track_iss(ISS_URL)


KeyboardInterrupt: 

In [6]:
import time
import requests
import pandas as pd
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from IPython.display import clear_output
from bokeh.models import ColumnDataSource

ISS_URL = 'https://api.wheretheiss.at/v1/satellites/25544'
ORBIT_TIME_SECS = 5_520  # Time required for ~1 complete orbit of ISS.


def get_iss_telemetry(url):
    """Return DataFrame of current ISS telemetry from URL."""
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch ISS position from {url}. \
        Status code: {response.status_code}")
    data = response.json()
    telemetry = pd.DataFrame(data, index=[0])
    telemetry = telemetry.drop(['id', 'footprint', 'daynum',
                                'solar_lat', 'solar_lon'], axis=1)
    return telemetry


def update_plot(latitudes, longitudes, source):
    source.stream({'latitudes': [latitudes[-1]], 'longitudes': [longitudes[-1]]})


def track_iss(url, num_orbits=2, interval=3):
    """
    Plot current ISS location from URL and record and plot its track.

    Arguments:
       url = ISS telemetry URL -> string
       num_orbits = Number of ISS orbits to plot -> integer
       interval = Wait period (seconds) between calls to URL -> integer
   """

    num_pulls = int(ORBIT_TIME_SECS * num_orbits / interval)
    latitudes = []
    longitudes = []
    
    output_notebook()

    p = figure(title='International Space Station Tracking', width=700, height=500)
    p.circle(x='longitudes', y='latitudes', source=source, size=10, color='blue')

    show(p, notebook_handle=True)

    for _ in range(num_pulls):
        df = get_iss_telemetry(url).round(2)
        latitudes.append(df['latitude'].iloc[0])
        longitudes.append(df['longitude'].iloc[0])
        clear_output(wait=True)
        update_plot(latitudes, longitudes, source)
        push_notebook()

        time.sleep(interval)


source = ColumnDataSource({'latitudes': [], 'longitudes': []})
track_iss(ISS_URL)


KeyboardInterrupt: 